In [1]:
from nltk import bigrams, word_tokenize
from nltk.util import ngrams
t='I am a boy'
tk = word_tokenize(t)
bg = bigrams(tk) # 직전의 단어
ng = ngrams(tk, 3) # 특정영역
list(bg),list(ng)

([('I', 'am'), ('am', 'a'), ('a', 'boy')],
 [('I', 'am', 'a'), ('am', 'a', 'boy')])

SS<문장>SE : 문장의 시작과 끝 표현

In [2]:
data2 = ngrams(tk,2,pad_left=True, pad_right=True,left_pad_symbol="SS",right_pad_symbol="SE")
list(data2) # 인덱스로 바로 접근할 수 없다.

[('SS', 'I'), ('I', 'am'), ('am', 'a'), ('a', 'boy'), ('boy', 'SE')]

In [3]:
from nltk import ConditionalFreqDist
t = 'I am a boy'
t2 = 'you are a boy'
tk = word_tokenize(t)
ng2 = ngrams(tk,2,pad_left=True, pad_right=True,left_pad_symbol="SS",right_pad_symbol="SE")
fd = ConditionalFreqDist([(i[0],i[1])for i in ng2])


In [4]:
fd.conditions() # 정답 접근 정보

['SS', 'I', 'am', 'a', 'boy']

In [5]:
fd['SS']

FreqDist({'I': 1})

In [6]:
import nltk
nltk.download('movie_reviews')
nltk.download('punkt')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\82105\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\82105\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
from nltk.corpus import movie_reviews
data_l = []
n = 0
for i in movie_reviews.sents():
    bg2 = ngrams(i,2,pad_left=True, pad_right=True,left_pad_symbol="SS",right_pad_symbol="SE")
    data_l+=[t for t in bg2]
    n+=1
cfd = ConditionalFreqDist(data_l)
#print(n)
#len(data_l)

In [8]:
cfd["SS"].most_common(10)

[('the', 8071),
 ('.', 3173),
 ('it', 3136),
 ('i', 2471),
 ('but', 1814),
 ('and', 1735),
 ('he', 1672),
 ('in', 1659),
 ('this', 1651),
 ('there', 1298)]

In [9]:
cfd["the"].most_common(10)

[('film', 4542),
 ('movie', 2147),
 ('story', 985),
 ('most', 945),
 ('first', 902),
 ('same', 774),
 ('only', 665),
 ('end', 664),
 ('best', 642),
 ('audience', 620)]

In [10]:
cfd["movie"].most_common(10)

[('.', 802),
 (',', 659),
 ('is', 469),
 ("'", 234),
 ('that', 210),
 ('was', 123),
 ('and', 105),
 ('with', 87),
 ('to', 72),
 ('has', 69)]

In [11]:
cfd["."].most_common(10)

[('SE', 63404), ('"', 1854), (')', 535), ("'", 70), (']', 10), ("''", 3)]

In [12]:
from nltk.probability import ConditionalProbDist, MLEProbDist
cpd = ConditionalProbDist(cfd, MLEProbDist)

In [13]:
cpd['the'].prob('movie')

0.0280547243528597

In [14]:
cpd['movie'].prob('.')

0.13897071564720154

In [15]:
cpd['.'].prob('the')

0.0

In [16]:
import numpy as np
def s_sc_f(x): # 바이그램 언어 모형의 확률 연산 함수
    p = 0.0
    for i in range(len(x)-1):
        c = x[i]
        w = x[i+1]
        p+=np.log(cpd[c].prob(w) + np.finfo(float).eps)
    return np.exp(p)

In [17]:
test_data = ['the','movie','.'] # 실현 가능문장
s_sc_f(test_data)

0.003898785120601922

In [18]:
test_data2 = ['movie','.','the'] # 실현 불가능문장
s_sc_f(test_data2) # e-17 => 0.00000000000 .....~~~~거의 0에 가깝다.

3.085769765203191e-17

In [19]:
import random
random.seed(10)
cpd['SS'].generate() # 임의의 값을 만들어낸다.

'she'

In [20]:
cpd['she'].generate()

'and'

In [21]:
cpd['and'].generate()

'fine'

In [22]:
cpd['fine'].generate()

'effect'

In [23]:
cpd['effect'].generate()

';'

data = ''
data += 문자열
''.join([]) # 연결연산

문장의 생성 (조건부 확률을 기반한 바이그램 언어 모델)

In [24]:
st = 'SS'
all_str = []
while True: # 언제끝날지 모르고 조건을 알때
    import random
    random.seed(10)
    st = cpd[st].generate()
    all_str.append(st+' ')
    if st=='SE':
        all_str.pop()
        break
''.join(all_str)

"she wasn ' s first part of these guys catch a hard 2 is . "

In [25]:
from nltk.util import ngrams # 앞단어와 뒷단어를 갖고 있는 data생성용
from nltk import ConditionalFreqDist # 문맥별 단어 빈도수 측정 클래스
from nltk.probability import ConditionalProbDist # 조건부 확률 추정 클래스
from nltk.probability import MLEProbDist # 최대 우도 추정값 클래스

MLEProbDist().generate # 샘플추출 (임의의 값을 추출)

1. 데이터 수집

In [26]:
from nltk.corpus import movie_reviews
data = movie_reviews.sents() # 단어별 토큰화 된 문장들

2. 데이터 전처리
    - 토큰화
    - 정형화
    - 정규화

In [27]:
data_l=[] # 토큰화된 단어 조합.
for 문장 in data:
    bg = ngrams(문장,2,pad_left=True, pad_right=True,left_pad_symbol="SS",right_pad_symbol="SE")
    data_l+=[t for t in bg]

3. 모델 학습

In [28]:
cfd = ConditionalFreqDist(data_l)
cpd = ConditionalProbDist(cfd, MLEProbDist)

4. 검증(생략)

5. 최종동작
    - data별 추가적 처리가 필요하다
    - 자동 랜덤 문장 생성기

In [29]:
st = 'SS'
all_str = []
import random
random.seed(10) # 시드고정으로 인한 데이터고정
while True: # 언제끝날지 모르고 조건을 알때
    st = cpd[st].generate() # 임의로 샘플 추출
    all_str.append(st+' ') # 리스트로 기록
    if st=='SE':
        all_str.pop()
        break
생성된_data = ''.join(all_str) # 리스트의 내용을 이용하여 하나의 문자열로 정리

In [30]:
생성된_data

'she and fine effect ; frankly , and doing his son becomes shockingly lazy shortcut to her mother of " story " '